In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import minimize
from sklearn.datasets import make_regression

/Users/svkov42/miniconda3/envs/ds/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
n = 10000
m = 4
bias = 10
x, y, coef = make_regression(n, m - 1, bias=bias, coef=True)
coef = np.hstack(([bias], coef)).reshape(-1, 1)
y = y.reshape(-1, 1)
x_ones = np.ones((n, 1))
x = np.hstack((x_ones, x))
x.shape

(10000, 4)

## Задача

Научимся применять модель с любыми весами

In [3]:
def forward(X, w):
    return X @ w

In [4]:
w = np.zeros((m, 1))
print(f'Прогноз для нулей\n{forward(x, w)[:5].flatten()}')

w = np.ones((m, 1))
print(f'Прогноз для единиц\n{forward(x, w)[:5].flatten()}')

w = coef.copy()
print(f'Прогноз линейной регрессии\n{forward(x, w)[:5].flatten()}')

Прогноз для нулей
[0. 0. 0. 0. 0.]
Прогноз для единиц
[ 3.22252898  4.38089189  3.73077617 -0.01778321 -0.84497553]
Прогноз линейной регрессии
[ 111.18509369  178.48866008   87.0763532    -4.57872402 -123.59787986]


## Задача

Написать лосс-функцию для линейной регрессии

In [5]:
def loss(X, y, w):
    return np.linalg.norm(forward(X, w) - y) / len(y)

In [6]:
w = np.zeros((m, 1))
print(f'Лосс большой {loss(x, y, w):.2f}')

w = np.ones((m, 1))
print(f'Лосс поменбше {loss(x, y, w):.2f}')

w = coef.copy()
print(f'Лосс ваще маленький жесть {loss(x, y, w):.2f}')

Лосс большой 1.02
Лосс поменбше 1.01
Лосс ваще маленький жесть 0.00


## Задача

Написать градиент лосс-функции

Есть два варианта, выбираем какой больше нравится:

- Аналитический
- Численный

Формула для аналитического градиента:

$$
\frac{2}{N} X^T(\hat{y} - y)
$$

In [7]:
def grad_loss(X, y, w):
    return (2 / len(y)) * X.T @ (forward(X, w) - y)
    
def grad_norm(X, y, w):
    return np.linalg.norm(grad_loss(X, y, w))

In [8]:
w = np.zeros((m, 1))
print(f'Градиент большой {grad_loss(x, y, w)}')

w = np.ones((m, 1))
print(f'Градиент поменбше {grad_loss(x, y, w)}')

w = coef.copy()
print(f'Градиент ваще маленький жесть {grad_loss(x, y, w)}')

Градиент большой [[ -21.94330223]
 [-137.61294834]
 [-148.4793108 ]
 [ -21.86531903]]
Градиент поменбше [[ -19.90190644]
 [-135.62496489]
 [-146.45420539]
 [ -19.87218928]]
Градиент ваще маленький жесть [[-1.39244172e-16]
 [-5.21483041e-17]
 [ 2.91401846e-17]
 [ 7.64230034e-17]]


In [9]:
w = np.zeros((m, 1))
print(f'Норма градиента большая {grad_norm(x, y, w)}')

w = np.ones((m, 1))
print(f'Норма градиента поменбше {grad_norm(x, y, w)}')

w = coef.copy()
print(f'Норма градиента ваще маленький жесть {grad_norm(x, y, w)}')

Норма градиента большая 204.79997552777454
Норма градиента поменбше 201.57865751602148
Норма градиента ваще маленький жесть 1.6969976661026071e-16


In [10]:
def random_row(x ,y):
    row_num = np.random.randint(0, x.shape[0])
    random_x = x[row_num, :].reshape(1, -1)
    random_y = y[row_num, :].reshape(1, -1)
    return random_x, random_y

In [11]:
def sgd_full(x, y, w_start, n_iters=1000, lr=0.1, tol=0.0001):
    # Ваш код здесь
    i = 0
    while grad_norm(x, y, w_start) > tol and i < n_iters:
        x_k, y_k = random_row(x ,y)
        w_start = w_start - lr * grad_loss(x_k, y_k, w_start)
        i += 1
    return w_start

In [12]:
w = np.random.normal(size=(m, 1))
lr = 0.01

In [13]:
w = sgd_full(x, y, w, 10000, lr, tol=1e-9)
w

array([[10.        ],
       [68.9274355 ],
       [73.8098344 ],
       [11.53343213]])

In [14]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x, y)
model.coef_, model.intercept_

(array([[ 0.        , 68.9274355 , 73.8098344 , 11.53343213]]), array([10.]))

нолик появился, потому что там вектор единиц